# Getting Started with AWS Converse API


### Initialize bedrock-runtime

In [2]:
import boto3, json

session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

message_list = []

### CODE 1 - A basic call to the Converse API

In [3]:
initial_message = {
    "role": "user",
    "content": [
        { "text": "How are you today?" } 
    ],
}

message_list.append(initial_message)

response = bedrock.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=message_list,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0
    },
)

response_message = response['output']['message']
print(json.dumps(response_message, indent=4))

{
    "role": "assistant",
    "content": [
        {
            "text": "I'm doing well, thanks for asking! I'm an AI assistant created by Anthropic to be helpful, harmless, and honest."
        }
    ]
}


###  CODE 2 - Alternating user and assistant messages

####  Alternate between messages from the “user” and “assistant” roles.

In [4]:
message_list.append(response_message)
print(json.dumps(message_list, indent=4))

[
    {
        "role": "user",
        "content": [
            {
                "text": "How are you today?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "I'm doing well, thanks for asking! I'm an AI assistant created by Anthropic to be helpful, harmless, and honest."
            }
        ]
    }
]


### CODE 3 - Including an image in a message

![Alt Text](image.webp)

In [13]:
with open("image.webp", "rb") as image_file:
    image_bytes = image_file.read()
    
image_message = {
    "role": "user",
    "content": [
        { "text": "Image 1:" },
        {
            "image": {
                "format": "webp",
                "source": {
                    "bytes": image_bytes #no base64 encoding required!
                }
            }
        },
        { "text": "Please describe the image." }
    ],
}

message_list.append(image_message)

response = bedrock.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=message_list,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0
    },
)


response_message = response['output']['message']
print(json.dumps(response_message, indent=4))

message_list.append(response_message)

{
    "role": "assistant",
    "content": [
        {
            "text": "The image shows a miniature model of a house, likely a decorative or toy item. The house has a blue exterior with white window frames and a red tiled roof. It appears to be made of a ceramic or porcelain material. The miniature house is placed on a surface with some greenery and yellow flowers surrounding it, creating a whimsical and natural setting. The background is slightly blurred, drawing focus to the intricate details of the small house model."
        }
    ]
}


### CODE 4 - Setting a system prompt


In [14]:
summary_message = {
    "role": "user",
    "content": [
        { "text": "Can you please summarize our conversation so far?" } 
    ],
}

message_list.append(summary_message)


response = bedrock.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=message_list,
    system=[
        { "text": "Please respond to all requests in the style of a doctor." }
    ],
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0
    },
)

response_message = response['output']['message']
print(json.dumps(response_message, indent=4))

message_list.append(response_message)

print("Stop Reason:", response['stopReason'])
print("Usage:", json.dumps(response['usage'], indent=4))

{
    "role": "assistant",
    "content": [
        {
            "text": "Sure, here's a summary of our conversation so far:\n\nYou greeted me and asked how I was doing today. I responded that I'm an AI assistant created by Anthropic to be helpful, harmless, and honest.\n\nThen you shared an image of a miniature model of a house and asked me to describe it. I provided a detailed description, noting the blue exterior, white window frames, red tiled roof, and that it appears to be made of ceramic or porcelain material. I mentioned it was placed in a natural setting with greenery and yellow flowers surrounding it, creating a whimsical scene. The background was slightly blurred to draw focus to the intricate house model details.\n\nAfter describing the image, you asked me to summarize our conversation up to this point, which is what I've done here."
        }
    ]
}
Stop Reason: end_turn
Usage: {
    "inputTokens": 628,
    "outputTokens": 177,
    "totalTokens": 805
}
